# Avtomatizacija poteka

## Primer: reader paket

### Dodamo teste

Ustvarimo mapo tests, dodamo `__init__.py`

In [ ]:
# test_feed.py 
  
"""Tests for the reader.feed module"""
# Standard library imports
import os.path

# Third party imports
import pytest

# Reader imports
from reader import feed

# Current directory
HERE = os.path.dirname(__file__)


@pytest.fixture
def local_feed():
    """Use local file instead of downloading feed from web"""
    return os.path.join(HERE, "realpython_20180919.xml")


@pytest.fixture
def local_summary_feed():
    """Use local file instead of downloading feed from web"""
    return os.path.join(HERE, "realpython_descriptions_20180919.xml")


#
# Tests
#
def test_site(local_feed):
    """Test that we can read the site title and link"""
    expected = "Real Python (https://realpython.com/)"
    assert feed.get_site(url=local_feed) == expected


def test_article_title(local_feed):
    """Test that title is added at top of article"""
    article_id = 0
    title = feed.get_titles(url=local_feed)[article_id]
    article = feed.get_article(article_id, url=local_feed)

    assert article.strip("# ").startswith(title)


def test_article(local_feed):
    """Test that article is returned"""
    article_id = 2
    article_phrases = [
        "logging.info('This is an info message')",
        "By using the `level` parameter",
        "  * `level`: The root logger",
    ]
    article = feed.get_article(article_id, url=local_feed)

    for phrase in article_phrases:
        assert phrase in article


def test_titles(local_feed):
    """Test that titles are found"""
    titles = feed.get_titles(url=local_feed)

    assert len(titles) == 20
    assert titles[0] == "Absolute vs Relative Imports in Python"
    assert titles[9] == "Primer on Python Decorators"


def test_invalid_article_id(local_feed):
    """Test that invalid article ids are handled gracefully"""
    article_id = "wrong"
    with pytest.raises(ValueError):
        feed.get_article(article_id, url=local_feed)

In [ ]:
# test_viewer.py
"""Tests for the reader.viewer module"""

# Third party imports
import pytest

# Reader imports
from reader import viewer


#
# Tests
#
def test_show(capsys):
    """Test that show adds information to stdout"""
    text = "Lorem ipsum dolor sit amet"
    viewer.show(text)
    stdout, stderr = capsys.readouterr()
    assert stderr == ""

    # It's ok if the viewer adds some information
    assert text in stdout


def test_show_list(capsys):
    """Test that show_list shows a list of items with an ID"""
    site = "Real Python"
    things = ["pathlib", "data classes", "python 3.7", "decorators"]
    viewer.show_list(site, things)
    stdout, stderr = capsys.readouterr()
    assert stderr == ""

    # Site name is shown in header
    lines = stdout.split("\n")
    assert site in lines[0]

    # Each thing is listed preceded by a number
    for thing, line in zip(things, lines[1:]):
        line_parts = line.split()
        assert line_parts[0].isnumeric()
        assert thing in line

> Dodamo še dve datoteke s podatki iz mape Test/data

V requirements_dev.txt dodamo naslednje:

    pytest
    memory_profiler
    coverage
    pytest-cov
    pylint

### Running Your Tests From Visual Studio Code

If you’re using the Microsoft Visual Studio Code IDE, support for unittest, nose, and pytest execution is built into the Python plugin.

If you have the Python plugin installed, you can set up the configuration of your tests by opening the Command Palette with `Ctrl+Shift+P` and typing “Python test”. You will see a range of options:
    
Choose `Debug All Unit Tests`, and VSCode will then raise a prompt to configure the test framework. Click on the cog to select the test runner (unittest) and the home directory (.).

Once this is set up, you will see the status of your tests at the bottom of the window, and you can quickly access the test logs and run the tests again by clicking on these icons:

### Ustvarimo Makefile

Konfiguracija pylinta:

    # izdelamo config file
    pylint --generate-rcfile > .pylintrc

V configu spremenimo: 

    [MESSAGES CONTROL] disable=missing-docstring
    [BASIC] good-names=f

Makefile: 
    
    TO_CHECK = ./reader/ 

    flake8:
        ### FLAKE8 ###
        flake8 $(TO_CHECK) --count --select=E9,F63,F7,F82 --show-source --statistics --exclude .git,__pycache__,.venv
        flake8 $(TO_CHECK) --count --exit-zero --max-complexity=10 --max-line-length=127 --statistics --exclude .git,__pycache__,.venv

    pylint:
        ### PYLINT ###
        pylint --rcfile .pylintrc $(TO_CHECK)

    test:
        ### Pytest ###
        pytest


    check_all: flake8 pylint test

### Add code to Github repository

- Ustvarimo repositoriji
- dodamo ssh public key v repozitorij
- dodamo .gitignore
- dodmao kodo na github

https://github.com/github/gitignore/blob/master/Python.gitignore

    git init
    git remote add origin git@github.com:leon11s/reader.git
    git add . 
    git commit -m "first commit"
    git branch -M main
    git push -u origin main

### Github actions

https://docs.github.com/en/free-pro-team@latest/actions/guides/building-and-testing-python

This guide shows you how to build, test, and publish a Python package.

GitHub-hosted runners have a tools cache with pre-installed software, which includes Python and PyPy. You don't have to install anything! For a full list of up-to-date software and the pre-installed versions of Python and PyPy, see "Specifications for GitHub-hosted runners".

#### Starting with the Python workflow template

GitHub provides a Python workflow template that should work for most Python projects. This guide includes examples that you can use to customize the template. For more information, see the Python workflow template.

To get started quickly, add the template to the .github/workflows directory of your repository.

To get started quickly, add the template to the `.github/workflows/upload_workflow.yml` directory of your repository.

    name: Python package

    on: [push]

    jobs:
      build:
        runs-on: ubuntu-latest
        steps:
        - uses: actions/checkout@v2
        - name: Set up Python 3.8
          uses: actions/setup-python@v2
          with:
            # Semantic version range syntax or exact version of a Python version
            python-version: '3.8'
            # Optional - x64 or x86 architecture, defaults to x64
            architecture: 'x64'
        # You can test your matrix by printing the current Python version
        - name: Display Python version
          run: python -c "import sys; print(sys.version)"
        - name: Install dependencies
          run: |
            python -m pip install --upgrade pip
            pip install flake8 pytest
            if [ -f requirements.txt ]; then pip install -r requirements.txt; fi
        - name: Lint with flake8
          run: |
            # stop the build if there are Python syntax errors or undefined names
            flake8 ./reader/ --count --select=E9,F63,F7,F82 --show-source --statistics
            # exit-zero treats all errors as warnings. The GitHub editor is 127 chars wide
            flake8 ./reader/ --count --exit-zero --max-complexity=10 --max-line-length=127 --statistics
        - name: Test with pytest
          run: |
            pytest


To use a pre-installed version of Python or PyPy on a GitHub-hosted runner, use the setup-python action. This action finds a specific version of Python or PyPy from the tools cache on each runner and adds the necessary binaries to PATH, which persists for the rest of the job. If a specific version of Python is not pre-installed in the tools cache, the setup-python action will download and set up the appropriate version from the python-versions repository.

Using the setup-python action is the recommended way of using Python with GitHub Actions because it ensures consistent behavior across different runners and different versions of Python. If you are using a self-hosted runner, you must install Python and add it to PATH. For more information, see the setup-python action.